In [ ]:
import shubhlipi as sh
from test_data_types import MainDataType, ResultDataType, MetaDataType
import os
import jinja2
import dotenv


In [ ]:
# Clearing the output folder
if os.path.isdir("../api/jee/process_data/out/"):
    sh.delete_folder("../api/jee/process_data/out/")
if sh.cmd("cd ../api/ && go run app.go test")[0] != 0:
    raise Exception("Error")
data = MainDataType(**sh.load_json(sh.read("../api/jee/process_data/out/data.json")))
result = ResultDataType(
    **sh.load_json(sh.read("../api/jee/process_data/out/result.json"))
)

meta_data_json = sh.load_json(sh.read("../api/jee/process_data/out/meta_data.json"))
metadata = MetaDataType(
    ApplicationNumber=meta_data_json["key"],
    Name=meta_data_json["name"],
    RollNumber=meta_data_json["roll"],
    DOB=meta_data_json["DOB"],
    TestDate=meta_data_json["date"],
    TestTime=meta_data_json["time"],
)


In [ ]:
# functions to hydrate the markdown data


def render_jinja_temp(fl: str, **o):
    return (
        jinja2.Environment(loader=jinja2.FileSystemLoader("./template/"))
        .get_template(fl)
        .render(**o)
    )


def get_result_markdown(data: MainDataType, result: ResultDataType) -> str:
    # We can also pass the data as a dict or even directly as shown below
    return render_jinja_temp(
        "result_temp.md.j2", data=data, result=result, metadata=metadata.dict()
    )


def get_questions_markdown(data: MainDataType, sc: ResultDataType) -> str:
    return render_jinja_temp("questions_temp.md.j2", data=data)


In [ ]:
# Deletiing the result folder
if os.path.exists("./result"):
    sh.delete_folder("./result")
sh.makedir("./result")


# Saving the result
sh.write("./result/result.md", get_result_markdown(data, result))
# Saving the questions
sh.write("./result/questions.md", get_questions_markdown(data, result))


In [ ]:
# Loading env variables
if os.path.isfile("../api/.env"):
    dotenv.load_dotenv("../api/.env")

SAVE_RESULT_TO_DATABASE = False
# The above will be true if we want to save the result and the data into the database
# This example data will be used a sample result for the frontend
if SAVE_RESULT_TO_DATABASE:
    print("Saving to database")
    from deta import Deta

    deta = Deta(sh.from_base64(os.getenv("DETA_KEY")))

    deta.Base("info").put(
        {"name": "", "DOB": "", "date": "", "time": "", "roll": ""}, "sample_result"
    )

    deta.Base("data").put(
        {
            "result": result.dict(),
            "data": data.dict(),
        },
        "sample_result",
    )
    print("Saved to database")
